In [ ]:
!pip install spotipy
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import requests
import numpy as np
from math import pi, ceil
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cid = '<your client id>'
secret = '<your client secret'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read'

In [ ]:
def get_features_from_playlist(user='', playlist_id=''):
    '''
    Takes in a user_id and a playlist_id and returns a dataframe of a user's playlist songs
    '''
    df_result = pd.DataFrame()
    track_list = ''
    uploader_list = []
    added_ts_list = []
    artist_list = []
    title_list = []

    more_songs = True #As long as there is tracks not fetched from API, continue looping
    offset_index = 0
    
    if playlist_id != '' and user == '':
        print("Enter username for playlist")
        return

    while more_songs:
        songs = sp.user_playlist_tracks(user, playlist_id=playlist_id, offset=offset_index)

        for song in songs['items']:
            #join track ids to a single string as an input parameter for audio_features function
            track_list += song['track']['id'] +','

            #get the time when the song was added
            added_ts_list.append(song['added_at'])

            #get the title of the song
            title_list.append(song['track']['name'])

            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
            
            #get user who added song in the playlist, catering for collaboration playlists
            uploader_list.append(song['added_by']['id'])

        #get the track features and append into a dataframe
        track_features = sp.audio_features(track_list[:-1])
        df_temp = pd.DataFrame(track_features)
        df_result = df_result.append(df_temp)
        track_list = ''

        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
            print('Progress: ' + str(offset_index) + ' of '+ str(songs['total']))

    #add the timestamp added, title and artists of a song
    df_result['added_at'], df_result['song_title'], df_result['artists'] = added_ts_list, title_list, artist_list
    
    #add upload user if in a playlist
    df_result['uploaded_by'] = uploader_list
        
    print('--- COMPLETED ---')
    
    return df_result

In [ ]:
#Get playlist data
df_playlist = get_features_from_playlist(user='Spotify', playlist_id='37i9dQZF1DX3ZKargsaFiQ')
#Artist 1 are the even rows, Artist 2 are the odd rows
artist1 = df_playlist.iloc[::2]
artist1['battle_artist'] = 'Erykah Badu'

artist2 = df_playlist.iloc[1::2]
artist2['battle_artist'] = 'Jill Scott'


concat_data = [artist1, artist2]
data_set = pd.concat(concat_data)
print(data_set)